# Анализ  перевозок по файлам из http://brokenstone.ru/

In [1]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd

In [2]:
# Загрузка файла
file = "br.xlsx"
df = pd.read_excel(file)

In [3]:
# Список дорог
railways = (
    'ВСБ', 'ГОР', 'ДВС', 'ЖДЯ', 'ЗАБ', 'ЗСБ', 
    'КБШ', 'КЛГ', 'КРС', 'МСК', 'ОКТ', 'ПРВ', 
    'СВР', 'СЕВ', 'СКВ', 'ЮВС', 'ЮУР')

In [4]:
# Исключения
place_of_calculation = ["Перевозки ОАО РЖД"]

# Фильтры
type_of_transportation = ["Внутр. перевозки", "внутренние"]
cargo_group = ["Минерально-строит."]
cargo_OKVED = ["Разработка карьеров"]
type_of_van = ["Полувагоны", "Полувагоны ост."]
departure_railways = ["СКВ","ОКТ","ЮУР"]

In [5]:
# Костыли из-за разных версий файлов
df["Груз"] = df["Груз"].fillna("")
df["Субъект РФ отправления"] = df["Область отправления"].fillna("")
df["Субъект РФ назначения"] = df["Область назначения"].fillna("")
df["Расстояние"] = df["Вагоно-км"].fillna(0)
df["Провозная плата"] = df["Тариф"].fillna(0)

df['Объем перевозок тонн'].replace(regex=True, inplace=True, to_replace=r',', value=r'.')
df['Объем перевозок тонн'] = df['Объем перевозок тонн'].astype('float32')
df['Объем перевозок тонн'].round(decimals=2)

df['Оператор'] = np.where(df['Оператор'] == 'вне аренды', df['Собственник вагона по ЕГРПО'], df['Оператор'])
df['Оператор'] = np.where(df['Оператор'] == '0', df['Собственник вагона по ЕГРПО'], df['Оператор'])
df['Оператор'] = np.where(df['Оператор'] == '10', df['Собственник вагона по ЕГРПО'], df['Оператор'])
df['Оператор'] = np.where(df['Оператор'] == '4', df['Собственник вагона по ЕГРПО'], df['Оператор'])
df['Оператор'] = np.where(df['Оператор'] == '#', df['Собственник вагона по ЕГРПО'], df['Оператор'])
df['Оператор'] = np.where(df['Оператор'] == 'неизвестен', df['Собственник вагона по ЕГРПО'], df['Оператор'])

df['Грузополучатель ОКПО'].replace(regex=True, inplace=True, to_replace=r"'", value=r'')
df['Грузополучатель наим по ЕГРПО'] = np.where(df['Грузополучатель наим по ЕГРПО'].isna(), df['Грузополучатель ОКПО'], df['Грузополучатель наим по ЕГРПО'])

**Предобработка данных**

In [6]:
# Применение фильтров и исключений
df = df.loc[df['Вид перевозки'].isin(type_of_transportation)]
#df = df.loc[df['Группа груза'].isin(cargo_group)]
#df = df.loc[df['Груз ОКВЭД'].isin(cargo_OKVED)]
df = df.loc[df['Род вагона'].isin(type_of_van)]
#df = df.loc[~df['Признак места расчета'].isin(place_of_calculation)]

In [7]:
# Преобразование наименований организаций
def clearing_organizational_form(dataframe):
    dataframe.replace(regex=True, inplace=True, to_replace=r'АО "ДВ ТРАНСПОРТНАЯ ГРУППА"   АО "ДВТГ"', value=r'ДВТГ')
    dataframe.replace(regex=True, inplace=True, to_replace=r'^И.П.', value=r'ИП')
    dataframe.replace(regex=True, inplace=True, to_replace=r'  ', value=r' ')
    dataframe.replace(regex=True, inplace=True, to_replace=r'(?i)Общество с ограниченной ответственностью', value=r'ООО')
    dataframe.replace(regex=True, inplace=True, to_replace=r'(?i)^Акционерное общество', value=r'АО')
    dataframe.replace(regex=True, inplace=True, to_replace=r'(?i)Государственное унитарное предприятие', value=r'ГУП')
    dataframe.replace(regex=True, inplace=True, to_replace=r'(?i)Открытое акционерное общество', value=r'ОАО')
    dataframe.replace(regex=True, inplace=True, to_replace=r'(?i)Закрытое акционерное общество', value=r'ЗАО')
    dataframe.replace(regex=True, inplace=True, to_replace=r'(?i)Публичное акционерное общество', value=r'ПАО')
    dataframe.replace(regex=True, inplace=True, 
                      to_replace=r'(?i)(^OOO |^АО |^ООО |^ОАО |^ЗАО |^ОАО|^ООО|^ПАО | ООО$| АО$| ЗАО$| ОАО$| ООО$| ПАО$|[^\w\d\s+-.])', 
                      value=r'')
    dataframe.replace(regex=True, inplace=True, to_replace=r'(?i)ФЕДЕРАЛЬНАЯ ГРУЗОВАЯ КОМПАНИЯ', value=r'ФГК')
    dataframe.replace(regex=True, inplace=True, to_replace=r'(?i)Новая перевозочная компания', value=r'НПК')
    dataframe.replace(regex=True, inplace=True, to_replace=r'(?i)Холдинговая компания', value=r'ХК')
    dataframe.replace(regex=True, inplace=True, to_replace=r'ЭЙ-СИ-РЕЙЛ', value=r'Эй-Си-Рейл')
    dataframe.replace(regex=True, inplace=True, to_replace=r'Кузбасстрансцемент', value=r'КузбассТрансЦемент')
    dataframe.replace(regex=True, inplace=True, to_replace=r'Новотранс ХК', value=r'ХК Новотранс')
    dataframe.replace(regex=True, inplace=True, to_replace=r'ХК НОВОТРАНС', value=r'ХК Новотранс')
    dataframe.replace(regex=True, inplace=True, to_replace=r'^\s', value=r'')

clearing_organizational_form(df['Собственник вагона по ЕГРПО'])
clearing_organizational_form(df['Арендатор вагона по ГВЦ'])
clearing_organizational_form(df['Оператор'])
clearing_organizational_form(df['Плательщик жд тарифа'])
clearing_organizational_form(df['Грузоотправитель наим по ЕГРПО'])
clearing_organizational_form(df['Грузополучатель наим по ЕГРПО'])

In [8]:
# Владелец вагона (арендатор или собственник)
#df['Арендатор вагона по ГВЦ'].apply(lambda x: x.lower())
#df['Владелец вагона'] = np.where(df['Арендатор вагона по ГВЦ'] == 'вне аренды', df['Собственник вагона по ЕГРПО'], df['Арендатор вагона по ГВЦ'])

In [9]:
# Столбцы, которые необходимо оставить
df_new = df[[
    'Номер документа', 'Дата отправления', 'Груз', 'Дорога отправления',
    'Субъект РФ отправления', 'Станция отправления РФ', 'Грузоотправитель наим по ЕГРПО',
    'Дорога назначения', 'Субъект РФ назначения', 'Станция назначения РФ', 'Грузополучатель наим по ЕГРПО',
    'Грузополучатель ОКПО', 'Плательщик жд тарифа', 'Расстояние', 'Объем перевозок тонн', 'Провозная плата', 
    'Оператор', 'Номер вагона']].reset_index(drop=True)
df_new = df_new.loc[df_new['Дорога отправления'].isin(departure_railways)]

**Анализ по собственникам вагонов**

In [10]:
# Сводная таблица по собственникам вагонов
vag_pivot_table = df.pivot_table(
    columns=['Дорога отправления','Дорога назначения'], 
    values='Номер документа', 
    aggfunc=len, 
    index='Оператор', 
    fill_value=0)

In [11]:
# Топ собственников вагонов по дороге отправления
def vag_top(departure_railway, count: int = 30):
    df_railway = df[df['Дорога отправления'] == departure_railway]
    vag_railway_pivot_table = df_railway.pivot_table(
        columns=['Дорога назначения'], 
        index='Оператор', 
        values='Объем перевозок тонн', 
        aggfunc=len, 
        fill_value=0, 
        margins=True)
    vag_railway_top = vag_railway_pivot_table.sort_values(by='All', ascending=False).head(count+1)
    vag_railway_top = vag_railway_top.drop(index=['All'])
    return vag_railway_top

In [12]:
skv_vag_top = vag_top(departure_railway='СКВ', count=30)
okt_vag_top = vag_top(departure_railway='ОКТ', count=30)
yur_vag_top = vag_top(departure_railway='ЮУР', count=30)

**Анализ по грузополучателям**

In [13]:
# Топ грузополучателей по дороге назначения
def consignees_top(destinations_railway, count: int = 30):
    df_railway = df[df['Дорога назначения'] == destinations_railway]
    consignee_railway_pivot_table = df_railway.pivot_table(
        columns=['Дорога отправления'], 
        index='Грузополучатель наим по ЕГРПО', 
        values='Объем перевозок тонн', 
        aggfunc=np.sum, 
        fill_value=0, 
        margins=True)
    consignee_railway_top = consignee_railway_pivot_table.sort_values(by='All', ascending=False).head(count+1)
    consignee_railway_top = consignee_railway_top.drop(index=['All'])
    consignee_railway_top = consignee_railway_top.loc[:, (consignee_railway_top != 0).any(axis=0)]
    return consignee_railway_top

In [14]:
msk_consignees_top = consignees_top(destinations_railway='МСК', count=30)
gor_consignees_top = consignees_top(destinations_railway='ГОР', count=30)

**Запись в файл**

In [15]:
# Генерация имени конечного файла
date_min = df['Дата отправления'].min()
date_max = df['Дата отправления'].max()
name_file_output = f'Перевозки + анализ {date_min}-{date_max}'
print(date_min, date_max)

01.01.2022 31.01.2022


In [16]:
# Запись в файл
writer = pd.ExcelWriter(f'{name_file_output}.xlsx')
df_new.to_excel(writer, 'rzd', index=False)
vag_pivot_table.to_excel(writer, 'vag')
skv_vag_top.to_excel(writer, 'skv_vag')
okt_vag_top.to_excel(writer, 'okt_vag')
msk_consignees_top.to_excel(writer, 'to_msk')
gor_consignees_top.to_excel(writer, 'to_gor')
writer.save()